In [10]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Large, MobileNetV3Small, VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split 
import shutil



def prepare_data(input_dir, output_dir, test_size=0.2):
    # Filter out only the actual class directories (exclude 'train' and 'val')
    classes = [cls for cls in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, cls)) and cls not in ['train', 'val']]

    for cls in classes:
        cls_path = os.path.join(input_dir, cls)
        if not os.path.isdir(cls_path):
            continue

        images = os.listdir(cls_path)
        train_images, val_images = train_test_split(images, test_size=test_size, random_state=1337)

        train_dir = os.path.join(output_dir, 'train', cls)
        val_dir = os.path.join(output_dir, 'val', cls)

        # Ensure directories exist
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)

        for img in train_images:
            img_path = os.path.join(cls_path, img)
            try:
                shutil.copy(img_path, os.path.join(train_dir, img))
            except PermissionError:
                print(f"Permission error while copying {img_path}")
                continue

        for img in val_images:
            img_path = os.path.join(cls_path, img)
            try:
                shutil.copy(img_path, os.path.join(val_dir, img))
            except PermissionError:
                print(f"Permission error while copying {img_path}")
                continue

# Function to Train and Evaluate the Model
def train_and_evaluate(pretrained_model, model_name, output_dir):
    # Load base model
    base_model = pretrained_model(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False

    # Build the classifier
    classifier = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    print(f"Training {model_name}...")
    print(classifier.summary())

    # Data augmentation for the training set
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2]
    )

    # Only rescale for the test set
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    # Load training and testing data
    training_set = train_datagen.flow_from_directory(
        os.path.join(output_dir, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_set = test_datagen.flow_from_directory(
        os.path.join(output_dir, 'val'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    # Train the model
    history = classifier.fit(
        training_set,
        steps_per_epoch=training_set.samples // training_set.batch_size,
        epochs=20,
        validation_data=test_set,
        validation_steps=test_set.samples // test_set.batch_size,
        callbacks=[early_stopping, reduce_lr]
    )

    # Evaluate the model
    predictions = classifier.predict(test_set, steps=test_set.samples // test_set.batch_size)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_set.classes[:len(y_pred)]

    # Calculate metrics
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')

    print(f"Results for {model_name}:")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Classification report
    report = classification_report(y_true, y_pred, target_names=list(training_set.class_indices.keys()))
    print("Classification Report:")
    print(report)

    return history





def train_and_evaluate_from_scratch(model_name, output_dir):
    # Build the CNN model from scratch
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    print(f"Training {model_name} from scratch...")
    print(model.summary())

    # Data augmentation for the training set
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2]
    )

    # Only rescale for the test set
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    # Load training and testing data
    training_set = train_datagen.flow_from_directory(
        os.path.join(output_dir, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_set = test_datagen.flow_from_directory(
        os.path.join(output_dir, 'val'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    # Train the model
    history = model.fit(
        training_set,
        steps_per_epoch=training_set.samples // training_set.batch_size,
        epochs=20,
        validation_data=test_set,
        validation_steps=test_set.samples // test_set.batch_size,
        callbacks=[early_stopping, reduce_lr]
    )

    # Evaluate the model
    predictions = model.predict(test_set, steps=test_set.samples // test_set.batch_size)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_set.classes[:len(y_pred)]

    # Calculate metrics
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')

    print(f"Results for {model_name}:")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Classification report
    report = classification_report(y_true, y_pred, target_names=list(training_set.class_indices.keys()))
    print("Classification Report:")
    print(report)

    return history

In [5]:
# Specify input output for train test split directory

print(os.getcwd())  
input_dir = r'/home/danodoms/repos/npk-classification/dataset_balanced'
output_dir = r'/home/danodoms/repos/npk-classification/dataset_balanced/train_test_split'

# input_dir = os.path.abspath('../dataset_balanced')
# output_dir = os.path.abspath('./dataset_exp/train_test_split')

# Define the train-test split ratio (e.g., 20% test data)
test_ratio = 0.2

# Prepare the dataset
prepare_data(input_dir, output_dir)

/home/danodoms/repos/npk-classification/FINAL


In [3]:
# Call the function for MobileNetV2
history_mobilenet = train_and_evaluate(MobileNetV2, "MobileNetV2", output_dir)

I0000 00:00:1736071082.502327  325868 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1983 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:10:00.0, compute capability: 7.5


Training MobileNetV2...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,468 (9.24 MB)

...

 1/33 ━━━━━━━━━━━━━━━━━━━━ 5:12 10s/step - accuracy: 0.1250 - loss: 3.1995

I0000 00:00:1736071095.761830  325952 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


33/33 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.5243 - loss: 1.2847 - val_accuracy: 0.8633 - val_loss: 0.3967 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.7812 - loss: 0.5496

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.7812 - loss: 0.5496 - val_accuracy: 0.8828 - val_loss: 0.3801 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 747ms/step - accuracy: 0.8384 - loss: 0.4230 - val_accuracy: 0.8984 - val_loss: 0.3184 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.8125 - loss: 0.4807 - val_accuracy: 0.8867 - val_loss: 0.3410 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 744ms/step - accuracy: 0.8729 - loss: 0.3836 - val_accuracy: 0.9141 - val_loss: 0.2710 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.8750 - loss: 0.5151 - val_accuracy: 0.9062 - val_loss: 0.2619 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 743ms/step - accuracy: 0.8872 - loss: 0.2737 - val_accuracy: 0.9023 - val_loss: 0.2930 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.8125 - loss: 0.2616 - val_accura

In [6]:
# Call the function for ResNet50
history_resnet = train_and_evaluate(ResNet50, "ResNet50", output_dir)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training ResNet50...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,500 (90.98 MB)

 Trainable params: 262,788 (1.00 MB)

...

17/33 ━━━━━━━━━━━━━━━━━━━━ 10s 642ms/step - accuracy: 0.2600 - loss: 1.8277

2025-01-05 18:05:46.343955: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[8,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-05 18:05:46.723161: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[8,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,128,28,28]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivati

33/33 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.2670 - loss: 1.7068 - val_accuracy: 0.2539 - val_loss: 1.3426 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.2500 - loss: 1.3791

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.2500 - loss: 1.3791 - val_accuracy: 0.2695 - val_loss: 1.3389 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 770ms/step - accuracy: 0.3154 - loss: 1.3565 - val_accuracy: 0.3789 - val_loss: 1.3083 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 0.1875 - loss: 1.3814 - val_accuracy: 0.3906 - val_loss: 1.3100 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 790ms/step - accuracy: 0.3496 - loss: 1.3429 - val_accuracy: 0.2656 - val_loss: 1.2992 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.4375 - loss: 1.2963 - val_accuracy: 0.2539 - val_loss: 1.2998 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 767ms/step - accuracy: 0.3766 - loss: 1.3122 - val_accuracy: 0.3594 - val_loss: 1.2675 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.3750 - loss: 1.3112 - val_accu

In [7]:
# Call the function for EfficientNetB0
history_efficientnet = train_and_evaluate(EfficientNetB0, "EfficientNetB0", output_dir)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training EfficientNetB0...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,055 (16.08 MB)

 Trainable params: 164,484 (642.52 KB)

...

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.2485 - loss: 1.4614 - val_accuracy: 0.2461 - val_loss: 1.3941 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.1875 - loss: 1.4063

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.1875 - loss: 1.4063 - val_accuracy: 0.2500 - val_loss: 1.3911 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 727ms/step - accuracy: 0.2252 - loss: 1.4022 - val_accuracy: 0.2500 - val_loss: 1.3869 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.2188 - loss: 1.3927 - val_accuracy: 0.2578 - val_loss: 1.3863 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 758ms/step - accuracy: 0.2579 - loss: 1.3856 - val_accuracy: 0.2461 - val_loss: 1.3873 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.3750 - loss: 1.3826 - val_accuracy: 0.2500 - val_loss: 1.3881 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 27s 811ms/step - accuracy: 0.2508 - loss: 1.3896 - val_accuracy: 0.2578 - val_loss: 1.3862 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.3438 - loss: 1.3848 - val_accur

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [6]:
# Call the function for MobileNetV3Large
history_mobileNetV3Large = train_and_evaluate(MobileNetV3Large,"MobileNetV3Large", output_dir)

I0000 00:00:1736072648.279400  339420 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1916 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:10:00.0, compute capability: 7.5


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training MobileNetV3Large...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobileNetV3Large (Functional)   │ (None, 7, 7, 960)      │     2,996,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 960)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       123,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,119,876 (11.90 MB)

...

 1/33 ━━━━━━━━━━━━━━━━━━━━ 7:46 15s/step - accuracy: 0.2188 - loss: 1.6957

I0000 00:00:1736072670.597915  339835 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


33/33 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.2415 - loss: 1.5410 - val_accuracy: 0.2461 - val_loss: 1.3708 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0938 - loss: 1.4311

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.0938 - loss: 1.4311 - val_accuracy: 0.2539 - val_loss: 1.3675 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 747ms/step - accuracy: 0.2692 - loss: 1.3906 - val_accuracy: 0.3203 - val_loss: 1.3563 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.2500 - loss: 1.4086 - val_accuracy: 0.3438 - val_loss: 1.3552 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 741ms/step - accuracy: 0.3051 - loss: 1.3613 - val_accuracy: 0.3516 - val_loss: 1.3523 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.2500 - loss: 1.3889 - val_accuracy: 0.4180 - val_loss: 1.3474 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 753ms/step - accuracy: 0.3220 - loss: 1.3634 - val_accuracy: 0.4219 - val_loss: 1.3353 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.3125 - loss: 1.4016 - val_accu

In [7]:
# Call the function for MobileNetV3Small
history_mobileNetV3Small = train_and_evaluate(MobileNetV3Small,"MobileNetV3Small", output_dir)

4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training MobileNetV3Small...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,013,492 (3.87 MB)

 Trainable params: 74,372 (290.52 KB)

...

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.2471 - loss: 1.4461 - val_accuracy: 0.2461 - val_loss: 1.3813 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2188 - loss: 1.4518

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.2188 - loss: 1.4518 - val_accuracy: 0.2578 - val_loss: 1.3785 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 741ms/step - accuracy: 0.2575 - loss: 1.3917 - val_accuracy: 0.2461 - val_loss: 1.3748 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.2500 - loss: 1.3791 - val_accuracy: 0.2500 - val_loss: 1.3748 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 730ms/step - accuracy: 0.2831 - loss: 1.3815 - val_accuracy: 0.2539 - val_loss: 1.3752 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.2812 - loss: 1.3745 - val_accuracy: 0.2461 - val_loss: 1.3747 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 723ms/step - accuracy: 0.2456 - loss: 1.3876 - val_accuracy: 0.3203 - val_loss: 1.3730 - learning_rate: 5.0000e-04
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.4062 - loss: 1.3724 - val_ac

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [9]:
# Call the function for VGG16
history_vgg16 = train_and_evaluate(VGG16,"VGG16", output_dir)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Training VGG16...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,780,868 (56.38 MB)

 Trainable params: 66,180 (258.52 KB)

...

30/33 ━━━━━━━━━━━━━━━━━━━━ 2s 693ms/step - accuracy: 0.3385 - loss: 1.4011

2025-01-05 18:45:17.696712: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[8,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,3,224,224]{3,2,1,0}, f32[64,3,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-05 18:45:17.881928: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[8,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivat

33/33 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.3506 - loss: 1.3809 - val_accuracy: 0.5078 - val_loss: 0.9334 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 7s 240ms/step - accuracy: 0.6250 - loss: 0.9436

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 112ms/step - accuracy: 0.6250 - loss: 0.9436 - val_accuracy: 0.5000 - val_loss: 0.9273 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 789ms/step - accuracy: 0.5478 - loss: 0.9382 - val_accuracy: 0.6719 - val_loss: 0.7486 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.5938 - loss: 0.8086 - val_accuracy: 0.6484 - val_loss: 0.7567 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 772ms/step - accuracy: 0.6479 - loss: 0.7772 - val_accuracy: 0.7734 - val_loss: 0.6436 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.6562 - loss: 0.8069 - val_accuracy: 0.7812 - val_loss: 0.6499 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 762ms/step - accuracy: 0.6746 - loss: 0.7353 - val_accuracy: 0.7500 - val_loss: 0.6266 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.7188 - loss: 0.7721 - val_accu

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report
import os
import numpy as np



historyFromScratch = train_and_evaluate_from_scratch("from_scratch", output_dir)

Training from_scratch from scratch...


/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,476 (42.61 MB)

...

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 678ms/step - accuracy: 0.3839 - loss: 1.7634

2025-01-05 19:31:24.488820: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,222,222]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,224,224]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-05 19:31:24.602133: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,109,109]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,111,111]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend

33/33 ━━━━━━━━━━━━━━━━━━━━ 36s 852ms/step - accuracy: 0.3874 - loss: 1.7465 - val_accuracy: 0.6328 - val_loss: 0.7306 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.5938 - loss: 0.8805

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.5938 - loss: 0.8805 - val_accuracy: 0.6445 - val_loss: 0.7214 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 729ms/step - accuracy: 0.6068 - loss: 0.7546 - val_accuracy: 0.7344 - val_loss: 0.5639 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.6562 - loss: 0.6641 - val_accuracy: 0.7031 - val_loss: 0.5848 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 727ms/step - accuracy: 0.6798 - loss: 0.7097 - val_accuracy: 0.6797 - val_loss: 0.6531 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.5938 - loss: 0.7785 - val_accuracy: 0.7070 - val_loss: 0.6050 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 733ms/step - accuracy: 0.7159 - loss: 0.6278 - val_accuracy: 0.7969 - val_loss: 0.4950 - learning_rate: 5.0000e-04
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.7500 - loss: 0.6004 - val_accu

In [13]:
# Call the function for ResNet50V2
from tensorflow.keras.applications import ResNet50V2

history_resnet50v2 = train_and_evaluate(ResNet50V2,"ResNet50V2", output_dir)

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training ResNet50V2...


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,827,588 (90.90 MB)

 Trainable params: 262,788 (1.00 MB)

...

None
Found 1064 images belonging to 4 classes.
Found 268 images belonging to 4 classes.


/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.6499 - loss: 0.8989 - val_accuracy: 0.9023 - val_loss: 0.2495 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.8125 - loss: 0.6238

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.8125 - loss: 0.6238 - val_accuracy: 0.9062 - val_loss: 0.2613 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 752ms/step - accuracy: 0.8518 - loss: 0.3798 - val_accuracy: 0.8711 - val_loss: 0.2912 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.8125 - loss: 0.4373 - val_accuracy: 0.8672 - val_loss: 0.2966 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 738ms/step - accuracy: 0.8532 - loss: 0.3716 - val_accuracy: 0.9023 - val_loss: 0.2248 - learning_rate: 5.0000e-04
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.8750 - loss: 0.2824 - val_accuracy: 0.8984 - val_loss: 0.2478 - learning_rate: 5.0000e-04
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 754ms/step - accuracy: 0.8782 - loss: 0.3028 - val_accuracy: 0.9375 - val_loss: 0.1857 - learning_rate: 5.0000e-04
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.8125 - loss: 0.3957

In [14]:
# Call the function for InceptionV3
from tensorflow.keras.applications import InceptionV3

history_inceptionV3 = train_and_evaluate(InceptionV3,"InceptionV3", output_dir)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training InceptionV3...


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,065,572 (84.17 MB)

 Trainable params: 262,788 (1.00 MB)

...

None
Found 1064 images belonging to 4 classes.
Found 268 images belonging to 4 classes.


/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.4556 - loss: 1.3406 - val_accuracy: 0.8047 - val_loss: 0.5392 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.6250 - loss: 0.6914

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.6250 - loss: 0.6914 - val_accuracy: 0.8203 - val_loss: 0.5437 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 761ms/step - accuracy: 0.7248 - loss: 0.6429 - val_accuracy: 0.8008 - val_loss: 0.4604 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.8125 - loss: 0.4999 - val_accuracy: 0.7891 - val_loss: 0.4656 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 748ms/step - accuracy: 0.7627 - loss: 0.5370 - val_accuracy: 0.8906 - val_loss: 0.3492 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - accuracy: 0.8125 - loss: 0.6060 - val_accuracy: 0.8789 - val_loss: 0.3559 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 784ms/step - accuracy: 0.8109 - loss: 0.4647 - val_accuracy: 0.8477 - val_loss: 0.3688 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - accuracy: 0.8125 - loss: 0.4619 - val_accura

In [15]:
# Call the function for EfficientNetV2B0
from tensorflow.keras.applications import EfficientNetV2B0

history_efficientNetV2B0 = train_and_evaluate(EfficientNetV2B0,"EfficientNetV2B0", output_dir)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training EfficientNetV2B0...


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,083,796 (23.21 MB)

 Trainable params: 164,484 (642.52 KB)

...

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 54s 918ms/step - accuracy: 0.2421 - loss: 1.4502 - val_accuracy: 0.2500 - val_loss: 1.3895 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 6:32 12s/step - accuracy: 0.1250 - loss: 1.3383

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 105ms/step - accuracy: 0.1250 - loss: 1.3383 - val_accuracy: 0.2539 - val_loss: 1.3893 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 795ms/step - accuracy: 0.2556 - loss: 1.3929 - val_accuracy: 0.2461 - val_loss: 1.3913 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.2188 - loss: 1.4132 - val_accuracy: 0.2500 - val_loss: 1.3886 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 771ms/step - accuracy: 0.2514 - loss: 1.3890 - val_accuracy: 0.2500 - val_loss: 1.3863 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.1875 - loss: 1.3867 - val_accuracy: 0.2461 - val_loss: 1.3863 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 729ms/step - accuracy: 0.2305 - loss: 1.3865 - val_accuracy: 0.2578 - val_loss: 1.3863 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.2188 - loss: 1.3862 - val_accu

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [16]:
# Call the function for EfficientNetV2B1
from tensorflow.keras.applications import EfficientNetV2B1

history_efficientNetV2B1 = train_and_evaluate(EfficientNetV2B1,"EfficientNetV2B1", output_dir)

28456008/28456008 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training EfficientNetV2B1...


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-b1 (Functional)  │ (None, 7, 7, 1280)     │     6,931,124 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,095,608 (27.07 MB)

 Trainable params: 164,484 (642.52 KB)

...

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.2294 - loss: 1.4589 - val_accuracy: 0.2539 - val_loss: 1.3872 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.3438 - loss: 1.3439

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - accuracy: 0.3438 - loss: 1.3439 - val_accuracy: 0.2578 - val_loss: 1.3905 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 27s 806ms/step - accuracy: 0.2415 - loss: 1.4032 - val_accuracy: 0.2539 - val_loss: 1.3861 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - accuracy: 0.2812 - loss: 1.3779 - val_accuracy: 0.2500 - val_loss: 1.3882 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 789ms/step - accuracy: 0.2519 - loss: 1.3870 - val_accuracy: 0.2422 - val_loss: 1.3863 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 0.1562 - loss: 1.3875 - val_accuracy: 0.2539 - val_loss: 1.3863 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 752ms/step - accuracy: 0.2498 - loss: 1.3863 - val_accuracy: 0.2578 - val_loss: 1.3862 - learning_rate: 5.0000e-04
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.2812 - loss: 1.3862 - val_a

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [17]:
# Call the function for NASNetMobile
from tensorflow.keras.applications import NASNetMobile

history_nasnetmobile = train_and_evaluate(NASNetMobile,"NASNetMobile", output_dir)

19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Training NASNetMobile...


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ nasnet_mobile (Functional)      │ (None, 7, 7, 1056)     │     4,269,716 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 1056)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │       135,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,405,528 (16.81 MB)

 Trainable params: 135,812 (530.52 KB)

...

Epoch 1/20


E0000 00:00:1736083428.578628  339832 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1736083428.729016  339832 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


33/33 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.6053 - loss: 0.9544 - val_accuracy: 0.8047 - val_loss: 0.4659 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.7188 - loss: 0.6211

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - accuracy: 0.7188 - loss: 0.6211 - val_accuracy: 0.8281 - val_loss: 0.4340 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 790ms/step - accuracy: 0.8303 - loss: 0.4658 - val_accuracy: 0.8750 - val_loss: 0.3390 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 0.7188 - loss: 0.5317 - val_accuracy: 0.8867 - val_loss: 0.3302 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 753ms/step - accuracy: 0.8218 - loss: 0.4224 - val_accuracy: 0.8594 - val_loss: 0.3371 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.9375 - loss: 0.3204 - val_accuracy: 0.8711 - val_loss: 0.3199 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 756ms/step - accuracy: 0.8626 - loss: 0.3515 - val_accuracy: 0.8672 - val_loss: 0.3390 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.8750 - loss: 0.3870 - val_accu

In [18]:
# Call the function for DenseNet121
from tensorflow.keras.applications import DenseNet121

history_densenet121 = train_and_evaluate(DenseNet121,"DenseNet121", output_dir)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training DenseNet121...


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,169,220 (27.35 MB)

 Trainable params: 131,716 (514.52 KB)

...

None
Found 1064 images belonging to 4 classes.
Found 268 images belonging to 4 classes.


/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.5184 - loss: 1.1046 - val_accuracy: 0.7422 - val_loss: 0.5367 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.5938 - loss: 0.7197

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step - accuracy: 0.5938 - loss: 0.7197 - val_accuracy: 0.7812 - val_loss: 0.5035 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 782ms/step - accuracy: 0.7791 - loss: 0.5494 - val_accuracy: 0.9180 - val_loss: 0.2830 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.7812 - loss: 0.6398 - val_accuracy: 0.9141 - val_loss: 0.2857 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 777ms/step - accuracy: 0.8614 - loss: 0.3733 - val_accuracy: 0.9219 - val_loss: 0.2430 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - accuracy: 0.9062 - loss: 0.3177 - val_accuracy: 0.9336 - val_loss: 0.2293 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 783ms/step - accuracy: 0.8642 - loss: 0.3596 - val_accuracy: 0.9297 - val_loss: 0.2244 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - accuracy: 0.9062 - loss: 0.3188 - val_accu

In [19]:
# Call the function for Xception
from tensorflow.keras.applications import Xception

history_xception = train_and_evaluate(Xception,"Xception", output_dir)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training Xception...


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_9      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,124,268 (80.58 MB)

 Trainable params: 262,788 (1.00 MB)

...

None
Found 1064 images belonging to 4 classes.
Found 268 images belonging to 4 classes.


/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.5442 - loss: 0.9803 - val_accuracy: 0.8477 - val_loss: 0.3512 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 5s 157ms/step - accuracy: 0.9062 - loss: 0.4269

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.9062 - loss: 0.4269 - val_accuracy: 0.8594 - val_loss: 0.3575 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 778ms/step - accuracy: 0.7993 - loss: 0.4862 - val_accuracy: 0.9102 - val_loss: 0.2409 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 1.0000 - loss: 0.1821 - val_accuracy: 0.8945 - val_loss: 0.2603 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 712ms/step - accuracy: 0.8549 - loss: 0.3796 - val_accuracy: 0.9297 - val_loss: 0.2143 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.9062 - loss: 0.3090 - val_accuracy: 0.9258 - val_loss: 0.2171 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 741ms/step - accuracy: 0.8707 - loss: 0.3496 - val_accuracy: 0.9297 - val_loss: 0.2159 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.9062 - loss: 0.3284 - val_accura

In [20]:
# Call the function for DenseNet169
from tensorflow.keras.applications import DenseNet169

history_densenet169 = train_and_evaluate(DenseNet169,"DenseNet169", output_dir)

51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Training DenseNet169...


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet169 (Functional)        │ (None, 7, 7, 1664)     │    12,642,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_10     │ (None, 1664)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │       213,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,856,516 (49.04 MB)

 Trainable params: 213,636 (834.52 KB)

...

None
Found 1064 images belonging to 4 classes.
Found 268 images belonging to 4 classes.


/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.5150 - loss: 1.2190 - val_accuracy: 0.8789 - val_loss: 0.3787 - learning_rate: 0.0010
Epoch 2/20
 1/33 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.6562 - loss: 0.8408

/home/danodoms/repos/npk-classification/env/lib64/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.6562 - loss: 0.8408 - val_accuracy: 0.8945 - val_loss: 0.3596 - learning_rate: 0.0010
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 765ms/step - accuracy: 0.8146 - loss: 0.4723 - val_accuracy: 0.8789 - val_loss: 0.3125 - learning_rate: 0.0010
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - accuracy: 0.7812 - loss: 0.4147 - val_accuracy: 0.8789 - val_loss: 0.3088 - learning_rate: 0.0010
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 757ms/step - accuracy: 0.8714 - loss: 0.3424 - val_accuracy: 0.9414 - val_loss: 0.2010 - learning_rate: 0.0010
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.8438 - loss: 0.3952 - val_accuracy: 0.9531 - val_loss: 0.1872 - learning_rate: 0.0010
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 746ms/step - accuracy: 0.8711 - loss: 0.3283 - val_accuracy: 0.9258 - val_loss: 0.1983 - learning_rate: 0.0010
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.9688 - loss: 0.1454 - val_accur